# <img src="https://img.icons8.com/bubbles/100/000000/3d-glasses.png" style="height:50px;display:inline"> EE 046746 - Technion - Computer Vision


## Homework 4 - Structure From Motion
---

### <img src="https://img.icons8.com/dusk/64/000000/python.png" style="height:50px;display:inline"> Python Libraries
---

* `numpy`
* `matplotlib`
* `opencv` (or `scikit-image`)
* `scikit-learn`
* `scipy`
* Anything else you need (`os`, `pandas`, `csv`, `json`,...)

In [1]:
import cv2 as cv
import numpy as np
import scipy.optimize
import numpy.linalg as la
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# for visualization windows to pop out in jupyter (kernel may require restart after using GUIs)
%matplotlib qt

### <img src="https://img.icons8.com/bubbles/50/000000/checklist.png" style="height:50px;display:inline"> Tasks
---
* In all tasks, you should document your process and results in a report file (which will be saved as `.pdf`). 
* You can reference your code in the report file, but no need for actual code in this file, the code is submitted in a seprate folder as explained above.

### <img src="https://img.icons8.com/external-itim2101-lineal-color-itim2101/64/000000/external-robot-engineering-itim2101-lineal-color-itim2101.png" style="height:50px;display:inline"> Introduction 
---
One of the major areas of computer vision is 3D reconstruction. Given several 2D images of an environment, can we recover the 3D structure of the environment, as well as the position of the camera/robot? This has many uses in robotics and autonomous systems, as understanding the 3D structure of the environment is crucial to navigation. You don't want your robot constantly bumping into walls, or running over human beings!

<center> <img src="https://research.qut.edu.au/qcr/wp-content/uploads/sites/305/2021/02/Challenge_summary_pic-768x513.jpg" style="height:300px">
<center> Image Source - <a href="https://research.qut.edu.au/qcr/2021/02/17/2nd-robotic-vision-scene-understanding-challenge-launched-cvpr2021-embodied-ai-workshop/"> CVPR 21 Embodied AI Workshop </a>

In Part 1, you will be writing a set of functions to generate a sparse point cloud for some test images we have provided to you. The test images are 2 renderings of a temple from two different angles. We have also provided you with a `npz` file containing good point correspondences between the two images. You will first write a function that computes the fundamental matrix between the two images. Then write a function that uses the epipolar constraint to find more point matches between the two images. Finally, you will write a function that will triangulate the 3D points for each pair of 2D point correspondences.

In Part 2, you will be writing a set of functions to calibrate a camera and project a 3D CAD model to a 2D image after estimating the camera pose. We have provided you with a `npz` file containing corresponding 2D-3D pairs. You will first write a function that estimates a camera matrix given 2D-3D calibration points. Then write a function to decompose the estimated camera matrix to intrinsic/extrinsic parameters. Finally, you will write a script to project the provided 3D CAD model and compare it to a given 2D image of an airplane.

### <img src="https://img.icons8.com/pastel-glyph/64/000000/pain-points.png" style="height:50px;display:inline"> Part 1 - Sparse Reconstruction 
---
In this section, you will be writing a set of function to compute the sparse reconstruction from two sample images of a temple. You will first estimate the Fundamental matrix, compute point correspondences, then plot the results in 3D.
It may be helpful to read through Section 1.5 right now. In Section 1.5 we ask you to write a testing script that will run your whole pipeline. It will be easier to start that now and add to it as you complete each of the questions one after the other.

#### 1.1 - Eight Point Algorithm
---
In this question, you're going to use the eight point algorithm which is covered in class to estimate the fundamental matrix. Please use the point correspondences provided in `data/some_corresp.npz`; you can load and view the contents of a `.npz` file as follows:
> ``data = np.load("./Users/yiftachedelstain/ee046746-computer-vision-private/HW/hw4_structure_from_motion/data/some_corresp.npz")``
<br> `` print(data.files)`` </br>

* Write the following function:

In [2]:
data = np.load("./data/some_corresp.npz")
# plot the points on im1 and im2
im1 = cv.imread("./data/im1.png")
im2 = cv.imread("./data/im2.png")
plt.imshow(im1)
plt.scatter(data['pts1'][:,0], data['pts1'][:,1], c='r', s=10)
plt.show()
plt.imshow(im2)
plt.scatter(data['pts2'][:,0], data['pts2'][:,1], c='r', s=10)
plt.show()

In [3]:
def eight_point(pts1, pts2, pmax):
    """
    Eight Point Algorithm
    [I] pts1, points in image 1 (Nx2 matrix)
        pts2, points in image 2 (Nx2 matrix)
        pmax, scalar value computed as max(H1,W1)
    [O] F, the fundamental matrix (3x3 matrix)
    """
    # Step 1: Normalize the points
    T = np.array([[1/pmax, 0, 0], [0, 1/pmax, 0], [0, 0, 1]])
    pts1_homog = np.concatenate([pts1, np.ones((pts1.shape[0], 1))], axis=1)
    pts2_homog = np.concatenate([pts2, np.ones((pts2.shape[0], 1))], axis=1)
    
    pts1_normalized = (T @ pts1_homog.T).T
    pts2_normalized = (T @ pts2_homog.T).T
    
    # Step 2: Construct the matrix A
    A = np.zeros((pts1.shape[0], 9))
    for i in range(pts1.shape[0]):
        x1, y1 = pts1_normalized[i, 0], pts1_normalized[i, 1]
        x2, y2 = pts2_normalized[i, 0], pts2_normalized[i, 1]
        A[i] = [x2*x1, x2*y1, x2, y2*x1, y2*y1, y2, x1, y1, 1]
    
    # Step 3: Compute the fundamental matrix F
    _, _, V = np.linalg.svd(A)
    F = V[-1].reshape(3, 3)
    
    # Step 4: Enforce the rank-2 constraint
    F = _singularize(F)
    
    # Step 5: Un-normalize the fundamental matrix
    F = T.T @ F @ T
    
    return F

def _singularize(F):
    U, S, V = np.linalg.svd(F)
    S[-1] = 0
    F = U @ np.diag(S) @ V
    return F


where `pts1` and `pts2` are $N \times 2$ matrices corresponding to the $(x,y)$ coordinates of the $N$ points in the first and second image respectively, and `pmax` is a scale parameter. Implementation tips:
* Normalize points and un-normalize $F$: You should scale the data by dividing each coordinate by $p_{\text{max}}$ (the maximum of the image's width and height) using a transformation matrix $T$. After computing $F$, you will have to "unscale" the fundamental matrix. If $p_{\text{norm}} = Tp$, then $F_{\text{unnorm}} = T^T F T$. Note that this scaling is slightly simpler than "centering" that you did in the lecture, but for the purpose of this assingment it should suffice.

* You must enforce the rank 2 constraint on $F$ before unscaling. Recall that a valid fundamental matrix $F$ will have all epipolar lines intersect at a certain point, meaning that there exists a non-trivial null space for $F$. In general, with real points, the eight-point solution for $F$ will not come with this condition. To enforce the rank 2 constraint, decompose $F$ with SVD to get the three matrices $U,\Sigma,V$ such that $F = U\Sigma V^T$. Then force the matrix to be rank 2 by setting the smallest singular value in $\Sigma$ to zero, giving you a new $\Sigma'$. Now compute the proper fundamental matrix with $F' = U\Sigma' V^T$.

* You may find it helpful to refine the solution by using local minimization. This probably won't fix a completely broken solution, but may make a good solution better by locally minimizing a geometric cost function. For this we have provided a helper function `refineF` taking in $F$ and the two sets of points, which you can call from `eight_point` before unscaling $F$.

* Remember that the x-coordinate of a point in the image is its column entry and y-coordinate is the row entry. Also note that eight-point is just a "figurative" name, it just means that you need at least 8 points; your algorithm should use an over-determined system ($N > 8$ points).

* To visualize the correctness of your estimated $F$, use the provided function `displayEpipolarF`, which takes in $F$, and the two images. This GUI lets you select a point in one of the images and visualize the corresponding epipolar line in the other image (Figure 1).

*  Please include in your report the recovered $F$ and the visualization of some epipolar lines (similar to Figure 1).

* Helper functions:

In [4]:
# helper function 1: singualrizes F using SVD
def _singularize(F):
    U, S, V = np.linalg.svd(F)
    S[-1] = 0
    F = U.dot(np.diag(S).dot(V))

    return F

# helper function 2.1: defines an objective function using F and the epipolar constraint
def _objective_F(f, pts1, pts2):
    F = _singularize(f.reshape([3, 3]))
    num_points = pts1.shape[0]
    hpts1 = np.concatenate([pts1, np.ones([num_points, 1])], axis=1)
    hpts2 = np.concatenate([pts2, np.ones([num_points, 1])], axis=1)
    Fp1 = F.dot(hpts1.T)
    FTp2 = F.T.dot(hpts2.T)

    r = 0
    for fp1, fp2, hp2 in zip(Fp1.T, FTp2.T, hpts2):
        r += (hp2.dot(fp1))**2 * (1/(fp1[0]**2 + fp1[1]**2) + 1/(fp2[0]**2 + fp2[1]**2))

    return r

# helper function 2.2: refines F using the objective from above and local optimization
def refineF(F, pts1, pts2):
    f = scipy.optimize.fmin_powell(
        lambda x: _objective_F(x, pts1, pts2), F.reshape([-1]),
        maxiter=100000,
        maxfun=10000
    )

    return _singularize(f.reshape([3, 3]))

* Visualization functions:

In [5]:
# helper function 3.1: derives the epipoles using the essential matrix
def _epipoles(E):
    U, S, V = np.linalg.svd(E)
    e1 = V[-1, :]
    U, S, V = np.linalg.svd(E.T)
    e2 = V[-1, :]

    return e1, e2

# helper function 3.2: GUI that uses F to draw the epipolar lines in I2 correponding to chosen pts in I1
def displayEpipolarF(I1, I2, F):
    e1, e2 = _epipoles(F)

    sy, sx, _ = I2.shape

    f, [ax1, ax2] = plt.subplots(1, 2, figsize=(12, 9))
    ax1.imshow(I1)
    ax1.set_title('Select a point in this image')
    ax1.set_axis_off()
    ax2.imshow(I2)
    ax2.set_title('Verify that the corresponding point \n is on the epipolar line in this image')
    ax2.set_axis_off()

    while True:
        plt.sca(ax1)
        x, y = plt.ginput(1, mouse_stop=2)[0]

        xc, yc = int(x), int(y)
        v = np.array([[xc], [yc], [1]])

        l = F @ v
        s = np.sqrt(l[0]**2 + l[1]**2)

        if s == 0:
            raise ValueError('Zero line vector in displayEpipolar')

        l = l / s
        if l[1] != 0:
            xs = 0
            xe = sx - 1
            ys = -(l[0] * xs + l[2]) / l[1]
            ye = -(l[0] * xe + l[2]) / l[1]
        else:
            ys = 0
            ye = sy - 1
            xs = -(l[1] * ys + l[2]) / l[0]
            xe = -(l[1] * ye + l[2]) / l[0]

        ax1.plot(x, y, '*', markersize=6, linewidth=2)  # Corrected markersize
        ax2.plot([xs, xe], [ys, ye], linewidth=2)
        plt.draw()


In [6]:
def test_eight_point():
    # Load the images and corresponding points
    data = np.load('data/some_corresp.npz')
    pts1 = data['pts1']  # Points in image 1
    pts2 = data['pts2']  # Points in image 2

    # Load images
    I1 = plt.imread('data/im1.png')
    I2 = plt.imread('data/im2.png')

    # Calculate pmax
    pmax = max(I1.shape[0], I1.shape[1])

    # Compute the Fundamental Matrix using the Eight Point Algorithm
    F = eight_point(pts1, pts2, pmax)

    # Print the computed Fundamental Matrix
    print("Computed Fundamental Matrix F:")
    print(F)

    # Visualize the Epipolar Lines using the displayEpipolarF function
    displayEpipolarF(I1, I2, F)

test_eight_point()

Computed Fundamental Matrix F:
[[ 3.56440672e-09 -5.92131870e-08 -1.65029959e-05]
 [-1.30829066e-07 -1.31095126e-09  1.12471525e-03]
 [ 3.04775126e-05 -1.08013400e-03 -4.16583180e-03]]


KeyboardInterrupt: 

* Example result:

<center> <img src = "assets/displayEpipolarF_example.jpg" style="width:75%">
<center> Figure 1 - Epipolar lines visualization from "displayEpipolarF"

#### 1.2 - Epipolar Correpondences
---
To reconstruct a 3D scene with a pair of two images, we need to find many point pairs. A point pair is two points (one in each image) that correspond to the same 3D scene point. With enough of these pairs, when we plot the resulting 3D points, we will have a rough outline of the 3D object. You found point pairs in HW1 using feature detectors and feature descriptors, and testing a point in one image with every single point in the other image. But here we can use the fundamental matrix to greatly simplify this search.

<center> <img src = "assets/epipolar_theory.jpg" style="width:50%">
<center> Figure 2 - Epipolar Geometry. Potential matches for $p$ lie on the epipolar line $l'$.

Recall from class that given a point $p$ in one image (the left view in Figure 2). Its corresponding 3D scene point $P$ could lie anywhere along the line from the camera center $C$ to the point $p$. This line, along with a second image's camera center $C'$ (the right view in Figure 2) forms a plane. This plane intersects with the image plane of the second camera, resulting in a line $l'$ in the second image which describes all the possible locations that $p$ may be found in the second image. Line $l'$ is the epipolar line, and we only need to search along this line to find a match for point $p$ found in the first image.

* Write the following function:

In [7]:
def epipolar_correspondences(I1, I2, F, pts1, window_size=5):
    """
    Epipolar Correspondences
    [I] I1, image 1 (H1xW1 matrix)
        I2, image 2 (H2xW2 matrix)
        F, fundamental matrix from image 1 to image 2 (3x3 matrix)
        pts1, points in image 1 (Nx2 matrix)
    [O] pts2, points in image 2 (Nx2 matrix)
    """
    pts2 = []
    half_window = window_size // 2

    for pt1 in pts1:
        x1, y1 = int(pt1[0]), int(pt1[1])
        p1 = np.array([x1, y1, 1])

        # Compute the epipolar line in the second image
        l = F @ p1
        l = l / np.sqrt(l[0]**2 + l[1]**2)  # Normalize the line

        best_pt2 = None
        min_ssd = float('inf')

        # Search along the epipolar line
        for x2 in range(half_window, I2.shape[1] - half_window):
            y2 = int(-(l[0] * x2 + l[2]) / l[1])
            if y2 < half_window or y2 >= I2.shape[0] - half_window:
                continue

            # Extract windows around pt1 in I1 and candidate pt2 in I2
            window1 = I1[y1 - half_window:y1 + half_window + 1, x1 - half_window:x1 + half_window + 1]
            window2 = I2[y2 - half_window:y2 + half_window + 1, x2 - half_window:x2 + half_window + 1]

            # Compute SSD (Sum of Squared Differences)
            ssd = np.sum((window1 - window2) ** 2)

            if ssd < min_ssd:
                min_ssd = ssd
                best_pt2 = [x2, y2]

        pts2.append(best_pt2)

    return np.array(pts2)

where `I1` and `I2` are two-view images, `F` is the fundamental matrix computed for the two images using your `eight_point` function, `pts1` is a $N \times 2$ matrix containing the $(x,y)$ points in the first image, and the function should return `pts2`, a $N \times 2$ matrix, which contains the corresponding points in the second image. Implementation tips:

* To match one point $p$ in image 1, use the fundamental matrix to estimate the corresponding epipolar line $l'$ and generate a set of candidate points in the second image.

* For each candidate points $p'$, a similarity score between $p$ and $p'$ is computed. The point among candidates with highest score is treated as epipolar correspondence.

* There are many ways to define the similarity between two points. Feel free to use whatever you want and **describe it in your write-up**. One possible solution is to select a small window of size $w$ around the point $p$. Then compare this target window to the window of the candidate point in the second image. For the images we gave you, simple Euclidean distance or Manhattan distance should suffice.

* Remember to take care of data type and index range. You can use the provided function `epipolarMatchGUI` to visually test your function. Your function does not need to be perfect, but it should get most easy points correct, like corners, dots etc.

* Please include a screenshot of `epipolarMatchGUI` running with your implementation of `epipolar_correspondences` (similar to Figure 3). Mention the similarity metric you decided to use. Also comment on any cases where your matching algorithm consistently fails, and why you might think this is.

* Visualization function:

In [8]:
# helper function 4: GUI that uses F, and the matching function to draw the epipolar correpondences on the epipolar lines
def epipolarMatchGUI(I1, I2, F):
    sy, sx, sd = I2.shape

    f, [ax1, ax2] = plt.subplots(1, 2, figsize=(12, 9))
    ax1.imshow(I1)
    ax1.set_title('Select a point in this image')
    ax1.set_axis_off()
    ax2.imshow(I2)
    ax2.set_title('Verify that the corresponding point \n is on the epipolar line in this image')
    ax2.set_axis_off()

    while True:
        plt.sca(ax1)
        x, y = plt.ginput(1, mouse_stop=2)[0]

        xc, yc = int(x), int(y)
        v = np.array([[xc], [yc], [1]])

        l = F @ v
        s = np.sqrt(l[0]**2 + l[1]**2)

        if s == 0:
            raise ValueError('Zero line vector in displayEpipolar')

        l = l / s
        if l[0] != 0:
            xs = 0
            xe = sx - 1
            ys = -(l[0] * xs + l[2]) / l[1]
            ye = -(l[0] * xe + l[2]) / l[1]
        else:
            ys = 0
            ye = sy - 1
            xs = -(l[1] * ys + l[2]) / l[0]
            xe = -(l[1] * ye + l[2]) / l[0]

        ax1.plot(x, y, '*', markersize=6, linewidth=2)
        ax2.plot([xs, xe], [ys, ye], linewidth=2)

        # Draw points
        pc = np.array([[xc, yc]])
        p2 = epipolar_correspondences(I1, I2, F, pc)
        ax2.plot(p2[0, 0], p2[0, 1], 'ro', markersize=8, linewidth=2)
        plt.draw()

In [9]:
def test_epipolar_correspondences():
    # Load the images and corresponding points
    data = np.load('data/some_corresp.npz')
    pts1 = data['pts1']  # Points in image 1
    pts2 = data['pts2']  # Points in image 2

    # Load images
    I1 = plt.imread('data/im1.png')
    I2 = plt.imread('data/im2.png')

    # Calculate pmax
    pmax = max(I1.shape[0], I1.shape[1])

    # Compute the Fundamental Matrix using the Eight Point Algorithm
    F = eight_point(pts1, pts2, pmax)

    # Visualize the Epipolar Correspondences using the GUI
    epipolarMatchGUI(I1, I2, F)

# Run the test
test_epipolar_correspondences()

KeyboardInterrupt: 

* Example result:

<center> <img src = "assets/epipolarMatchGUI_example.jpg" style="width:75%">
<center> Figure 3 - Epipolar Match visualization. A few errors are alright, but it should get most easy points correct (corners, dots, etc.)

#### 1.3 - Essential Matrix
---
In order to get the full camera projection matrices we need to compute the Essential matrix. So far, we have only been using the Fundamental matrix.

* Write the following function:

In [10]:
def essential_matrix(F, K1, K2):
    """
    Essential Matrix
    [I] F, the fundamental matrix (3x3 matrix)
        K1, camera matrix 1 (3x3 matrix)
        K2, camera matrix 2 (3x3 matrix)
    [O] E, the essential matrix (3x3 matrix)
    """
    E = K2.T @ F @ K1
    return E

In [11]:
def test_essential_matrix():
    # Load the images and corresponding points
    data = np.load('data/some_corresp.npz')
    pts1 = data['pts1']  # Points in image 1
    pts2 = data['pts2']  # Points in image 2
    
    # Load the intrinsic matrices
    intrinsics = np.load('data/intrinsics.npz')
    K1 = intrinsics['K1']
    K2 = intrinsics['K2']
    
    # Calculate pmax
    pmax = max(pts1.shape[0], pts1.shape[1])
    
    # Compute the Fundamental Matrix using the Eight Point Algorithm
    F = eight_point(pts1, pts2, pmax)
    
    # Compute the Essential Matrix
    E = essential_matrix(F, K1, K2)
    
    # Print the computed Essential Matrix
    print("Computed Essential Matrix E:")
    print(E)
    
# Run the test
test_essential_matrix()
    

Computed Essential Matrix E:
[[ 4.60261645e-02 -8.37143325e-01 -2.62254028e-01]
 [-1.73129035e+00 -1.81926284e-02  9.65216091e+00]
 [-8.88283922e-03 -9.77247828e+00 -1.64220428e-02]]


Where `F` is the Fundamental matrix computed between two images, `K1` and `K2` are the intrinsic camera matrices for the first and second image respectively (contained in `data/intrinsics.npz`), and `E` is the computed essential matrix. The intrinsic camera parameters are typically acquired through camera calibration. Refer to the class slides for the
relationship between the Fundamental matrix and the Essential matrix.

* Please include your estimated $E$ matrix for the temple image pair in the PDF report.

#### 1.4 - Triangulation
---
Write a function to triangulate pairs of 2D points in the images to a set of 3D points.

* Write the following function:

In [12]:
def triangulate(M1, pts1, M2, pts2):
    """
    Triangulation
    [I] M1, camera projection matrix 1 (3x4 matrix)
        pts1, points in image 1 (Nx2 matrix)
        M2, camera projection matrix 2 (3x4 matrix)
        pts2, points in image 2 (Nx2 matrix)
    [O] pts3d, 3D points in space (Nx3 matrix)
    """
    num_points = pts1.shape[0]
    pts3d = np.zeros((num_points, 3))
    
    for i in range(num_points):
        x1, y1 = pts1[i]
        x2, y2 = pts2[i]
        
        # Create the system of linear equations
        A = np.array([
            x1 * M1[2] - M1[0],
            y1 * M1[2] - M1[1],
            x2 * M2[2] - M2[0],
            y2 * M2[2] - M2[1]
        ])
        
        # Solve for the 3D point using SVD
        _, _, V = np.linalg.svd(A)
        X = V[-1]
        X = X / X[3]  # Normalize to make it a homogeneous coordinate
        
        pts3d[i] = X[:3]
    
    return pts3d
    

Where `pts1` and `pts2` are the $N\times 2$ matrices with the 2D image coordinates, `M1` and `M2` are the $3\times4$ camera projection matrices and `pts3d` is an $N\times 3$ matrix with the corresponding 3D points (in all cases, one point per row). Remember that you will need to multiply the given intrinsic matrices with your solution for the extrinsic camera matrices to obtain the final camera projection matrices. 

* For `M1` you can assume no rotation or translation, so the extrinsic matrix is just $\left[I \lvert 0\right]$. 

* For `M2`, pass the essential matrix to the provided function `camera2` to get four possible **extrinsic** matrices. You will need to determine which of these is the correct one to use. Refer to the class slides to remind yourself of the 4 possible camera layouts. The correct configuration is the one for which most of the 3D points are in front of both cameras (i.e. have a positive depth), and make up a reasonable recovered shape. To make sure you choose the right one, write a helper function that apply the extrinsics of camera 2 to the recovered 3D points and check that they are still in front of camera 1 (positive $Z$).

* Keep in mind to multiply the extrinsics matrices by the corresponding intrinsics before inputting them to `triangulate`.
* Once implemented, check the performance by looking at the re-projection error. To compute the re-projection error, project the estimated 3D points back to the image 1 and compute the mean Euclidean error between projected 2D points and the given `pts1`.

* **In your write-up**: Describe how you determined which extrinsic matrix is correct. Note that simply rewording the hint is not enough. Report your re-projection error using the given `pts1` and `pts2` in `data/some_corresp.npz`. If implemented correctly, the re-projection error should be less than 2 pixels.

* Helper functions:

In [13]:
# helper function 5: returns the 4 options for camera matrix M2 given the essential matrix
def camera2(E):
    U,S,V = np.linalg.svd(E)
    m = S[:2].mean()
    E = U.dot(np.array([[m,0,0], [0,m,0], [0,0,0]])).dot(V)
    U,S,V = np.linalg.svd(E)
    W = np.array([[0,-1,0], [1,0,0], [0,0,1]])

    if np.linalg.det(U.dot(W).dot(V))<0:
        W = -W

    M2s = np.zeros([3,4,4])
    M2s[:,:,0] = np.concatenate([U.dot(W).dot(V), U[:,2].reshape([-1, 1])/abs(U[:,2]).max()], axis=1)
    M2s[:,:,1] = np.concatenate([U.dot(W).dot(V), -U[:,2].reshape([-1, 1])/abs(U[:,2]).max()], axis=1)
    M2s[:,:,2] = np.concatenate([U.dot(W.T).dot(V), U[:,2].reshape([-1, 1])/abs(U[:,2]).max()], axis=1)
    M2s[:,:,3] = np.concatenate([U.dot(W.T).dot(V), -U[:,2].reshape([-1, 1])/abs(U[:,2]).max()], axis=1)

    return M2s

def find_correct_M2(M1, pts1, M2s, pts2, K1, K2):
    best_M2 = None
    max_positive_depth = 0
    
    for i in range(4):
        M2 = M2s[:, :, i]
        P = triangulate(M1, pts1, K2 @ M2, pts2)
        
        # Check how many points are in front of both cameras
        num_positive_depth = np.sum(P[:, 2] > 0)
        
        if num_positive_depth > max_positive_depth:
            max_positive_depth = num_positive_depth
            best_M2 = M2
    
    return best_M2


In [24]:
def compute_reprojection_error(pts3d, M1, M2, pts1, pts2):
    """
    Compute the re-projection error.
    [I] pts3d, 3D points in space (Nx3 matrix)
        M1, camera projection matrix 1 (3x4 matrix)
        M2, camera projection matrix 2 (3x4 matrix)
        pts1, original points in image 1 (Nx2 matrix)
        pts2, original points in image 2 (Nx2 matrix)
    [O] error, the re-projection error (scalar)
    """

    # Convert 3D points to homogeneous coordinates
    pts3d_homogeneous = np.hstack((pts3d, np.ones((pts3d.shape[0], 1))))

    # Project the 3D points back onto the image planes
    projected_pts1_homogeneous = (M1 @ pts3d_homogeneous.T).T
    projected_pts2_homogeneous = (M2 @ pts3d_homogeneous.T).T

    # Convert back to 2D coordinates by dividing by the third coordinate
    projected_pts1 = projected_pts1_homogeneous[:, :2] / projected_pts1_homogeneous[:, 2].reshape(-1, 1)
    projected_pts2 = projected_pts2_homogeneous[:, :2] / projected_pts2_homogeneous[:, 2].reshape(-1, 1)

    # Compute the Euclidean distance (re-projection error) between original and projected points
    error1 = np.linalg.norm(pts1 - projected_pts1, axis=1)
    print(f"Re-projection error 1: {error1.mean()}")
    error2 = np.linalg.norm(pts2 - projected_pts2, axis=1)
    print(f"Re-projection error 2: {error2.mean()}")

    # Calculate the mean re-projection error
    total_error = np.mean(np.hstack((error1, error2)))

    return total_error

def test_reprojection_error():
    # Load necessary data
    data = np.load('data/some_corresp.npz')
    pts1 = data['pts1']  # Points in image 1
    pts2 = data['pts2']  # Points in image 2
    intrinsics = np.load('data/intrinsics.npz')
    K1 = intrinsics['K1']
    K2 = intrinsics['K2']

    # Load images
    I1 = plt.imread('data/im1.png')
    I2 = plt.imread('data/im2.png')

    # Compute the Fundamental Matrix using the Eight Point Algorithm
    F = eight_point(pts1, pts2, max(I1.shape))

    # Compute the Essential Matrix
    E = essential_matrix(F, K1, K2)

    # Compute the first camera projection matrix M1
    M1 = K1 @ np.hstack((np.eye(3), np.zeros((3, 1))))

    # Get the four possible M2s
    M2s = camera2(E)

    # Find the correct M2
    M2 = find_correct_M2(M1, pts1, M2s, pts2, K1, K2)

    # Triangulate to find the 3D points
    pts3d = triangulate(M1, pts1, K2 @ M2, pts2)

    # Compute the re-projection error
    error = compute_reprojection_error(pts3d, M1, K2 @ M2, pts1, pts2)

    print(f"Mean Re-projection error: {error:.4f} pixels")

test_reprojection_error()

Re-projection error 1: 0.4694339612615537
Re-projection error 2: 0.46861357015407
Mean Re-projection error: 0.4690 pixels


#### 1.5 - Putting It All Together
---
You now have all the pieces you need to generate a full 3D reconstruction. Write a test script `test_temple_coords` that does the following:

* Load the two images and the point correspondences from `data/some_corresp.npz`.
* Run eight point to compute the fundamental matrix `F`.
* Load the points in image 1 contained in `data/temple_coords.npz` and run your `epipolar_correspondences` on them to get the corresponding points in image 2.
* Load `data/intrinsics.npz` and compute the essential matrix `E`.
* Compute the first camera projection matrix `M1` and use `camera2` to compute the four candidates for `M2`.
* Run your `triangulate` function using the four sets of camera matrix candidates (after scaling with the intrinsics), the points from `data/temple_coords.npz`, and their computed correspondences.
* Figure out the correct `M2` and the corresponding 3D points.
* Use matplotlib's scatter function to plot these point correspondences on screen.
* Report your computed extrinsic parameters (`R1`,`R2`,`t1`,`t2`) in your PDF.
* We will use your test script to run your code, so be sure it runs smoothly. In particular, use relative paths to load files, not absolute paths.
* **In your write-up**: Include 3 images of your final reconstruction of the points given in the file `data/temple_coords.npz`, from different angles as shown in Figure 4.

* Example result:

<center> <img src = "assets/pointcloud_example.jpg" style="width:75%">
<center> Figure 4 - Sample Reconstructions.

In [16]:
def test_temple_coords():
    # Load the images and corresponding points
    data = np.load('data/some_corresp.npz')
    pts1 = data['pts1']  # Points in image 1
    pts2 = data['pts2']  # Points in image 2
    temple_data = np.load('data/temple_coords.npz')
    temple_pts1 = temple_data['pts1']  # Points in image 1 for temple
    
    # Load intrinsics
    intrinsics = np.load('data/intrinsics.npz')
    K1 = intrinsics['K1']
    K2 = intrinsics['K2']

    # Load images
    I1 = plt.imread('data/im1.png')
    I2 = plt.imread('data/im2.png')

    # Compute the Fundamental Matrix using the Eight Point Algorithm
    F = eight_point(pts1, pts2, max(I1.shape))

    # Compute corresponding points in image 2 for the temple points
    temple_pts2 = epipolar_correspondences(I1, I2, F, temple_pts1)

    # Compute the Essential Matrix
    E = essential_matrix(F, K1, K2)

    # Compute the first camera projection matrix M1
    M1 = K1 @ np.hstack((np.eye(3), np.zeros((3, 1))))

    # Get the four possible M2s
    M2s = camera2(E)

    # Find the correct M2
    M2 = find_correct_M2(M1, temple_pts1, M2s, temple_pts2, K1, K2)

    # Triangulate to find the 3D points
    P = triangulate(M1, temple_pts1, K2 @ M2, temple_pts2)

    # Visualize the 3D points using matplotlib's scatter function
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(P[:, 0], P[:, 1], P[:, 2], c='r', marker='o')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')

    plt.show()

    # Report the extrinsic parameters of M2
    R2 = M2[:, :3]
    t2 = M2[:, 3]
    print("Extrinsic Parameters for M2:")
    print("R2:\n", R2)
    print("t2:\n", t2)

test_temple_coords()

Extrinsic Parameters for M2:
R2:
 [[ 9.65189813e-01 -2.84559523e-02  2.59997852e-01]
 [ 2.73497924e-02  9.99594929e-01  7.87192866e-03]
 [-2.60116537e-01 -4.87018080e-04  9.65577107e-01]]
t2:
 [-1.         -0.02745167  0.08201567]
